In [ ]:
import os
script_name = 'run_script.ipynb'
dir_path = os.path.dirname(os.path.realpath(script_name))
parent_path = os.path.dirname(dir_path)
savemodel_path = os.path.join(parent_path,'model')
loaddata_path = os.path.join(parent_path,'data')
headfile_path = os.path.join(parent_path,'headfile')

In [ ]:
import sys
sys.path.append(savemodel_path)
sys.path.append(loaddata_path)
sys.path.append(headfile_path)

In [ ]:
from load_data import Hologram_Dataset,ToTensor
from build_network import TEST_NET, TEST_NET2
from model_utils import load_model,save_model
from run_network import train, test,train_onebyone
import matplotlib.pyplot as plt
import numpy as np
import torch
import cv2 as cv
import os
from torch import nn, optim
from torchvision import transforms, utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from skimage.measure import compare_ssim as ssim
import csv
import time
%matplotlib inline

In [ ]:
path = os.path.join(loaddata_path,'dataset_training_2')
input_channel = 2
batch_size = 10
training_dataset = Hologram_Dataset(path, dataset_type = 'single',
                                  input_name='input_data',output_name='output_data', 
                                  transform=ToTensor())
training_dataloader = DataLoader(training_dataset, batch_size=batch_size,shuffle=True, num_workers=6)
training_datasize = len(training_dataset)
print(training_datasize)

In [ ]:
# training_dataset[42000]['input'].shape
plt.imshow(training_dataset[5056]['input'][1].numpy())

In [ ]:
device = torch.device("cuda:0")

In [ ]:
model = TEST_NET2(input_channel)
model.to(device)



criterion = nn.MSELoss()
lr_init = 1e-5
lr = lr_init
lr_stop = 1e-14
lr_decay = 0.1
optimizer = optim.Adam(model.parameters(), lr=lr)
num_epochs = 10000
start_epoch = 0

In [ ]:
# NET_NAME = 'TEST_NET'
# dirname = 'inputs_'+str(input_channel)+'dataset_'+str(training_datasize)+'_'+NET_NAME
# filename = 'inputs_'+str(input_channel)+'dataset_'+str(training_datasize)+'epoch_'+str(start_epoch)+'_'+NET_NAME+'.pth'

# load_model(model, path = os.path.join(savemodel_path,dirname), model_name=filename, mode = "single")

In [ ]:
NET_NAME = 'TEST_NET2'
dirname = 'inputs_'+str(input_channel)+'dataset_'+str(training_datasize)+'_'+NET_NAME
if not os.path.exists(os.path.join(savemodel_path,dirname)):
    os.makedirs(os.path.join(savemodel_path,dirname))
    
loss_store = []
loss_comparison_len = 100
start_ratio = 3
start_time = time.time()
for epoch in range(start_epoch,num_epochs):
    tmp_loss = train_onebyone(training_dataloader,model,criterion, optimizer,device)
    tmp_loss = tmp_loss.data[0]
    if len(loss_store) > start_ratio*loss_comparison_len:
        if np.mean(loss_store[-1*loss_comparison_len:]) < tmp_loss:
            loss_store = []
            lr = lr*lr_decay
            if lr < lr_stop:
                print('converge!!!!!!')
                break
            optimizer = optim.Adam(model.parameters(), lr=lr)
#         else:
    loss_store.append(tmp_loss)
    if epoch < 10: 
        print('Epoch[{}/{}], loss: {:.6f}, time elapsed: {:.2f}'
                  .format(epoch+1, num_epochs, tmp_loss, time.time()-start_time))
    elif (epoch+1) % 10 == 0:
        print('Epoch[{}/{}], loss: {:.6f}, time elapsed: {:.2f}'
                  .format(epoch+1, num_epochs, tmp_loss, time.time()-start_time))
        filename = 'inputs_'+str(input_channel)+'dataset_'+str(training_datasize)+'epoch_'+str(epoch)+'_'+NET_NAME+'.pth'
        save_model(model, optimizer, path = os.path.join(savemodel_path,dirname), filename=filename)
#     if (epoch+1) % 500 == 0:
#         filename = 'inputs_'+str(input_channel)+'dataset_'+str(training_datasize)+'epoch_'+str(epoch)+'_'+NET_NAME+'.pth'
#         save_model(model, optimizer, path = os.path.join(savemodel_path,dirname), filename=filename) 

# filename = 'inputs_'+str(input_channel)+'dataset_'+str(training_datasize)+'epoch_'+str(epoch)+'_'+NET_NAME+'.pth'
# save_model(model, optimizer, path = os.path.join(savemodel_path,dirname), filename=filename) 

In [ ]:
filename = 'inputs_'+str(input_channel)+'dataset_'+str(training_datasize)+'epoch_'+str(epoch)+'_'+NET_NAME+'.pth'
save_model(model, optimizer, path = os.path.join(savemodel_path,dirname), filename=filename) 

In [ ]:
test_path = os.path.join(loaddata_path,'dataset_testing_2')
testing_dataset = Hologram_Dataset(test_path, dataset_type = 'single',input_name='input_data',output_name='output_data',transform=ToTensor())
testing_datasize = len(testing_dataset)
testing_dataloader = DataLoader(testing_dataset, batch_size=testing_datasize,shuffle=False, num_workers=1)

In [ ]:
predict_test,gt_test = test(testing_dataloader,model,device)

In [ ]:
plt.imshow(predict_test[0,0])

In [ ]:
plt.imshow(gt_test[0,0])

In [ ]:
print(testing_dataset[1]['input'].shape)
plt.imshow(testing_dataset[0]['input'][1].numpy())

In [ ]:
lr